In [1]:
#import libraries used into
from xml.etree.ElementTree import tostring
from datetime import datetime
import pandas as pd
import numpy as np
import os

In [2]:
##First, the vars was created
fileam = "OT " + datetime.now().strftime('%d.%m.%Y') + ".XLSX"
filepm = "OT" + datetime.now().strftime('%d.%m.%Y') + "5 P.M.XLSX"
main_path = "C:\\Users\\mruiz\\Documents\\DB OT" 
current_month = datetime.now().strftime('%B')
main_path_ = os.listdir(main_path)

In [3]:
for item in main_path_:
    if item == fileam:
        item1 = item
        print("Los datos del día de hoy son : ", item1)
        break

data1 = main_path + "\\" + item1
data1excel = pd.read_excel(data1, sheet_name="Sheet1", header=0)

Los datos del día de hoy son :  OT 12.10.2022.XLSX


In [4]:
data1excel = data1excel.assign(Material=0)

In [13]:
data1excel = data1excel.drop('Material',axis=1)

Para convertir y eliminar el archivo creado desde to_csv usar el siguiente codigo: 
for item in main_path_:
    if item == file:
        item1 = item
        print("Los datos del día de hoy son : ", item1)
        break
"""try:
    os.remove(main_path + "\\daily_data.csv")
except OSError as e:
    print(f"Error:{ e.strerror}")"""

data1 = main_path + "\\" + item1
data1excel = pd.read_excel(data1, sheet_name="Sheet1", header=0)
data1excel_ = pd.DataFrame(data1excel)
"""path_csv = main_path + "\\daily_data.csv"
data1csv = data1excel_.to_csv(path_csv)
data1csv_ = pd.read_csv(path_csv)"""

In [122]:
data1excel['Material'] = np.where((data1excel.Clase_de_orden =='ZMCP') & (data1excel.Texto_breve.__contains__('EMERG' or 'REVI')),'CONTIENE MATERIAL',"DA")

In [78]:
print(data1excel.head())
print(data1excel.columns)

  Inicio_programado    Orden  Plan_mtto_preventivo Pto_tbjo_responsable  \
0        2024-07-26  4096274                8396.0             TECSER24   
1        2023-12-12  4093006                 626.0             TECSER17   
2        2023-10-12  4093001                 439.0              TECSER6   
3        2023-09-13  4093222                1522.0              TECSER2   
4        2023-09-11  4093049                 653.0              TECSER6   

  Status_del_sistema                               Texto_breve   Cliente  \
0          BLOQUEADA                SEGUIDORES DE LA CRUZ 10KW  303371.0   
1            ABIERTA  HD MIGUEL ALEMAN 8789 PTA 350 KW ACUMULA  300105.0   
2            ABIERTA  HD PIEDRAS NEGRAS 8638 PTA 350 KW ACUMUL  300105.0   
3            ABIERTA  HD INTERLOMAS 8762 PTA 250 KW ACUMULADOR  300105.0   
4            ABIERTA    HD SAN PEDRO 8652 PTA 350KW ACUMULADOR  300105.0   

     Denominación_de_objeto_técnico Número_de_serie   Equipo Clase_de_orden  \
0        SEGU

In [80]:
for item in main_path_:
    if item == 'Master_OT.xlsx':
        item2 = item
        print("La base de datos de nombres es : ", item2)
        break

data2 = main_path + "\\" + item2
data2excel = pd.read_excel(data2, sheet_name="Names", header=0)

La base de datos de nombres es :  Master_OT.xlsx


In [113]:
data3 = main_path + "\\POLITICA.XLSX"
data3excel = pd.read_excel(data3, sheet_name="POLITICAS", header=0)

In [114]:
print(data2excel.head())

      TecID          Short name  Estatus     Typetec       City
0   TECSER1     Jonatan Ledesma  FORANEO     Foraneo     Juarez
1  TECSER10      Jonathan Tovar    LOCAL  Correctivo  Monterrey
2  TECSER11  Victorio Rodriguez    LOCAL       Apoyo  Monterrey
3  TECSER13       Alan Resendiz    LOCAL  Correctivo  Monterrey
4  TECSER14       Alberto Gamez    LOCAL  Correctivo  Monterrey


In [123]:
result_df = pd.merge(data1excel,data2excel,left_on="Pto_tbjo_responsable",right_on="TecID",how="left")

In [124]:
result_df = pd.merge(result_df,data3excel,on="Equipo",how="left")

In [86]:
print(result_df.loc[0])

Inicio_programado                                          2024-07-26 00:00:00
Orden                                                                  4096274
Plan_mtto_preventivo                                                    8396.0
Pto_tbjo_responsable                                                  TECSER24
Status_del_sistema                                                   BLOQUEADA
Texto_breve                                         SEGUIDORES DE LA CRUZ 10KW
Cliente                                                               303371.0
Denominación_de_objeto_técnico                      SEGUIDORES DE LA CRUZ 10KW
Número_de_serie                                                            NaN
Equipo                                                                 12923.0
Clase_de_orden                                                            ZPM2
Material                                                     CONTIENE MATERIAL
TecID                                               

In [87]:
print(result_df.columns)
print(data1excel.shape[0])
print(result_df.shape[0])
print(result_df.head())


Index(['Inicio_programado', 'Orden', 'Plan_mtto_preventivo',
       'Pto_tbjo_responsable', 'Status_del_sistema', 'Texto_breve', 'Cliente',
       'Denominación_de_objeto_técnico', 'Número_de_serie', 'Equipo',
       'Clase_de_orden', 'Material', 'TecID', 'Short name', 'Estatus',
       'Typetec', 'City', 'Denominación de objeto técnico', 'Cliente actual',
       'Denominación de la ubicación técnica', 'Status del sistema',
       'Número de serie', 'TIENE POLITICA', 'TECH ASIGNADO AL EQUIPO',
       'gestora archivo anahi ', 'reportes archivo anahi ', 'Gestora',
       'Administradora Reportes', 'Tipo Equipo', 'Frecuencia',
       'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
       'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc',
       'Tiempo valido de prueba', 'Tecnicos para servicio',
       'Requerimientos adicionales', 'Correo solicitud acceso',
       'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales',
       'OC', 'FRECUENC

In [125]:
os.remove(main_path + "\\main.xlsx")
path = main_path + "\\main.xlsx"
result_df.to_excel(path, index=False,header=True)